# Preprocessing and define functions

In [1]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import random
import pickle
import datetime
import os
import torch.nn.functional as F
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation as R
import torch.nn.functional as F
from pytorch_metric_learning import miners, losses
import wandb

In [2]:
seed=999
os.environ["PL_GLOBAL_SEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sottile124 (pair_fragments). Use `wandb login --relogin` to force relogin


True

In [3]:
def use_GPU():
    """ This function activates the gpu 
    """
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(torch.cuda.get_device_name(0), "is available and being used")
    else:
        device = torch.device("cpu")
        print("GPU is not available, using CPU instead") 
    return device  


def center_in_origin(frag):
    """ This function normalize each fragment in (0,1)
    """
    min_vals, _ = torch.min(frag[:, 0:3], axis=0)
    max_vals, _ = torch.max(frag[:, 0:3], axis=0)
    frag[:, 0:3] = (frag[:, 0:3] - min_vals) / (max_vals - min_vals)
    
    return frag
    
def normalize(batch):
    """ This function apply center_in_origin() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(center_in_origin(element))
    out_tensor = torch.stack(out)    
    return out_tensor  



def translate_to_origin(frag):
    """ This function translate each fragment in the origin
    """
    frag[:,:3] -= torch.mean(frag[:,:3]) 
    return frag

def apply_translation(batch):
    """ This function apply translate_to_origin() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(translate_to_origin(element))
    out_tensor = torch.stack(out)    
    return out_tensor


def random_rotation(frag):

    randrot = (torch.rand(3)*360).tolist()
    r = R.from_euler('zyx', randrot, degrees=True)
    frag[:,:3] = torch.from_numpy(r.apply(frag[:,:3]))
    frag[:,3:6] = torch.from_numpy(r.apply(frag[:,3:6]))
    return frag

def apply_randomrotations(batch):
    """ This function apply random_rotation() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(random_rotation(element))
    out_tensor = torch.stack(out)    
    return out_tensor

In [4]:
# Code form the repository of PCT https://github.com/qq456cvb/Point-Transformers

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    return torch.sum((src[:, :, None] - dst[:, None]) ** 2, dim=-1)

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)


def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids

def sample_and_group(npoint, nsample, xyz, points):
    B, N, C = xyz.shape
    S = npoint 
    
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint]

    new_xyz = index_points(xyz, fps_idx) 
    new_points = index_points(points, fps_idx)

    dists = square_distance(new_xyz, xyz)  # B x npoint x N
    idx = dists.argsort()[:, :, :nsample]  # B x npoint x K

    grouped_points = index_points(points, idx)
    grouped_points_norm = grouped_points - new_points.view(B, S, 1, -1)
    new_points = torch.cat([grouped_points_norm, new_points.view(B, S, 1, -1).repeat(1, 1, nsample, 1)], dim=-1)
    return new_xyz, new_points


class Local_op(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        b, n, s, d = x.size()  # torch.Size([32, 512, 32, 6]) 
        x = x.permute(0, 1, 3, 2)
        x = x.reshape(-1, d, s)
        batch_size, _, N = x.size()
        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x))) # B, D, N
        x = torch.max(x, 2)[0]
        x = x.view(batch_size, -1)
        x = x.reshape(b, n, -1).permute(0, 2, 1)
        return x


class SA_Layer(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.q_conv.weight = self.k_conv.weight 
        self.v_conv = nn.Conv1d(channels, channels, 1)
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x_q = self.q_conv(x).permute(0, 2, 1) # b, n, c 
        x_k = self.k_conv(x)# b, c, n        
        x_v = self.v_conv(x)
        energy = x_q @ x_k # b, n, n 
        attention = self.softmax(energy)
        attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = x_v @ attention # b, c, n 
        x_r = self.act(self.after_norm(self.trans_conv(x - x_r)))
        x = x + x_r
        return x
    



    

class StackedAttention(nn.Module):
    def __init__(self, channels=256):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm1d(channels)
        self.bn2 = nn.BatchNorm1d(channels)

        self.sa1 = SA_Layer(channels)
        self.sa2 = SA_Layer(channels)
        self.sa3 = SA_Layer(channels)
        self.sa4 = SA_Layer(channels)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        # 
        # b, 3, npoint, nsample  
        # conv2d 3 -> 128 channels 1, 1
        # b * npoint, c, nsample 
        # permute reshape
        batch_size, _, N = x.size()

        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x)))

        x1 = self.sa1(x)
        x2 = self.sa2(x1)
        x3 = self.sa3(x2)
        x4 = self.sa4(x3)
        
        x = torch.cat((x1, x2, x3, x4), dim=1)

        return x

In [5]:
# the same as before but witch classical attention 
class SA_Layer_classic(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.q_conv.weight = self.k_conv.weight 
        self.v_conv = nn.Conv1d(channels, channels, 1)
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x_q = self.q_conv(x).permute(0, 2, 1) # b, n, c 
        x_k = self.k_conv(x)# b, c, n        
        x_v = self.v_conv(x)
        energy = x_q @ x_k # b, n, n 
        attention = self.softmax(energy)
        attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = x_v @ attention # b, c, n 
        x_r = self.act(self.after_norm(self.trans_conv(x_r)))
        
        return x_r


class StackedAttention_classic(nn.Module):
    def __init__(self, channels=256):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm1d(channels)
        self.bn2 = nn.BatchNorm1d(channels)

        self.sa1 = SA_Layer_classic(channels)
        self.sa2 = SA_Layer_classic(channels)
        self.sa3 = SA_Layer_classic(channels)
        self.sa4 = SA_Layer_classic(channels)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        # 
        # b, 3, npoint, nsample  
        # conv2d 3 -> 128 channels 1, 1
        # b * npoint, c, nsample 
        # permute reshape
        batch_size, _, N = x.size()

        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x)))

        x1 = self.sa1(x)
        x2 = self.sa2(x1)
        x3 = self.sa3(x2)
        x4 = self.sa4(x3)
        
        x = torch.cat((x1, x2, x3, x4), dim=1)

        return x





class Branch(nn.Module):
    def __init__(self):
        super().__init__()
        
        d_points = 7 # we have 7 features for each point
        self.conv1 = nn.Conv1d(d_points, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.gather_local_0 = Local_op(in_channels=128, out_channels=128)
        self.gather_local_1 = Local_op(in_channels=256, out_channels=256)
        self.pt_last = StackedAttention()###############################################################

        self.relu = nn.ReLU()
        self.conv_fuse = nn.Sequential(nn.Conv1d(1280, 1024, kernel_size=1, bias=False),
                                   nn.BatchNorm1d(1024),
                                   nn.LeakyReLU(negative_slope=0.2))

        
    def forward(self, x):
        xyz = x[..., :3]
        x = x.permute(0, 2, 1)
        batch_size, _, _ = x.size()
        x= x.double()
        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x))) # B, D, N
        x = x.permute(0, 2, 1)
        new_xyz, new_feature = sample_and_group(npoint=512, nsample=32, xyz=xyz, points=x)         
        feature_0 = self.gather_local_0(new_feature)
        feature = feature_0.permute(0, 2, 1)
        new_xyz, new_feature = sample_and_group(npoint=256, nsample=32, xyz=new_xyz, points=feature) 
        feature_1 = self.gather_local_1(new_feature)
        
        x = self.pt_last(feature_1)
        x = torch.cat([x, feature_1], dim=1)
        x = self.conv_fuse(x)
        x = torch.max(x, 2)[0] # Returns the maximum value of all elements in the input tensor. (2 elementes for each vector)
        x = x.view(batch_size, -1) # Returns a new tensor with the same data as the self tensor but of a different shape.
        
        return x
    

def apply_cosine_sim(A,B):
    cosine_similarities = F.cosine_similarity(A, B)

    min_value = torch.min(cosine_similarities)
    max_value = torch.max(cosine_similarities)
    scaled_similarities = (cosine_similarities - min_value) / (max_value - min_value)
    one_minus_scaled = 1 - scaled_similarities
    result_tensor = torch.cat((one_minus_scaled.view(-1, 1), scaled_similarities.view(-1, 1)), dim=1) 
    return result_tensor    


class ContrastiveLoss(torch.nn.Module):
    def __init__(self, m=2.0):
        super(ContrastiveLoss, self).__init__()
        self.m = m

    def forward(self, y1, y2, d):
        euc_dist = torch.nn.functional.pairwise_distance(y1, y2)

        if d.dim() == 0:  # Se d è uno scalare
            if d == 0:
                return torch.mean(torch.pow(euc_dist, 2))  # Distanza quadratica
            else:  # d == 1
                delta = self.m - euc_dist
                delta = torch.clamp(delta, min=0.0, max=None)
                return torch.mean(torch.pow(delta, 2))
        else:  # Se d è un tensore di valori 0 e 1
            is_same = d == 0
            is_diff = d == 1

            loss_same = torch.pow(euc_dist[is_same], 2).mean() if torch.any(is_same) else torch.tensor(0.0).to(euc_dist.device)
            loss_diff = torch.pow(torch.clamp(self.m - euc_dist[is_diff], min=0.0), 2).mean() if torch.any(is_diff) else torch.tensor(0.0).to(euc_dist.device)

            return (loss_same + loss_diff) / (1.0 + torch.any(is_same).float() + torch.any(is_diff).float())

In [9]:
# data loading
train = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\train_pair_dataset_REG.pt")
val = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\val_pair_dataset_REG.pt")
test = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\test_pair_dataset_REG.pt")

In [10]:
#let's find the largest clusters

threshold = 70

# Train
indices = [i for i, data in enumerate(train) if data[0].shape[0] > threshold]
count = len(indices)

# Val
indices_val = [i for i, data in enumerate(val) if data[0].shape[0] > threshold]
count_val = len(indices_val)

# Test
indices_test = [i for i, data in enumerate(test) if data[0].shape[0] > threshold]
count_test = len(indices_test)

print("Positions to remove (Train):", indices)
print("Positions to remove (Val):", indices_val)
print("Positions to remove (Test):", indices_test)


# We are removing the largest clusters for computational reasons
mask = torch.ones(train.shape[0], dtype=torch.bool)
mask[indices] = False
filtered_tensor = train[mask]
train = filtered_tensor

mask_val = torch.ones(val.shape[0], dtype=torch.bool)
mask_val[indices_val] = False
filtered_tensor_val = val[mask_val]
val = filtered_tensor_val

mask_test = torch.ones(test.shape[0], dtype=torch.bool)
mask_test[indices_test] = False
filtered_tensor_test = test[mask_test]
test= filtered_tensor_test   

print(train.shape)
print(val.shape)
print(test.shape)



Positions to remove (Train): [2, 14, 20, 22, 28, 30, 31, 35, 36, 39, 71, 78, 87, 91, 95, 109, 136, 148, 163, 167, 192, 197, 205, 209, 215, 217, 218, 222, 228, 241, 255, 261, 263, 265, 273, 284, 300, 308, 318, 337, 344, 346, 353, 361, 364, 369, 398, 420, 423, 427, 431, 433, 468, 469, 471, 474, 489, 494, 496, 507, 513, 516, 517, 538, 547, 551, 564, 577, 591, 595, 597, 600, 607, 613, 617, 633, 635, 644, 651, 652, 660, 664, 675, 689, 694, 699, 700, 707, 711, 721, 731, 735, 754, 757, 758, 763, 768, 771, 776, 783, 789, 813, 818, 831, 834, 837, 841, 844, 846, 874, 879, 896, 909, 917, 931, 943, 971, 977, 979, 988, 991, 994, 996, 999, 1042, 1045, 1050, 1057, 1077, 1078, 1079, 1084, 1100, 1109, 1118, 1126, 1127, 1129, 1130, 1131, 1143, 1150, 1156, 1188, 1217, 1219, 1227, 1254, 1274, 1284, 1290, 1295, 1299, 1316, 1324, 1334, 1336, 1337, 1347, 1353, 1356, 1390, 1392, 1412, 1418, 1460, 1463, 1466, 1469, 1474, 1483, 1489, 1498, 1499, 1504, 1505, 1507, 1514]
Positions to remove (Val): [6, 12, 13, 22,

In [11]:
def all_pos (vector):

    positions = np.arange(len(vector))  
    
    return positions, vector


def fixed_and_balanced (vector):

    labels = torch.Tensor(vector)
    num = min((labels==0).int().sum(), (labels==1).int().sum())
    
    zero_indices = (labels == 0).nonzero()[:num, 0]
    one_indices = (labels == 1).nonzero()[:num, 0]
    combined_indices = torch.sort(torch.cat((zero_indices, one_indices)))
    
    selected_labels = labels[combined_indices.values]
    return combined_indices.values.tolist(), selected_labels 

def fixed_and_unbalanced (vector):
     
    labels = torch.Tensor(vector)
    num_zeroes = (labels==0).int().sum() 
    num_ones = (labels==1).int().sum()
    num = min(num_zeroes, num_ones)
    
    extra_1= int((num_ones-num_zeroes)/3)
    extra_0= int((num_zeroes-num_ones)/3)
    
    zero_indices = (labels == 0).nonzero()[:num + extra_0, 0]
    one_indices = (labels == 1).nonzero()[:num + extra_1, 0]
    combined_indices = torch.sort(torch.cat((zero_indices, one_indices)))
    
    selected_labels = labels[combined_indices.values]
    return combined_indices.values.tolist(), selected_labels  

def sampled_balanced (vector):

    labels = torch.Tensor(vector)
    num_zeroes = (labels==0).int().sum() 
    num_ones = (labels==1).int().sum()
    num = min(num_zeroes, num_ones)

    zero_indices = (labels == 0).nonzero()[:num, 0]
    one_indices = (labels == 1).nonzero()[:num, 0]

    one_indices = one_indices[torch.randperm(one_indices.size(0))][:num]
    zero_indices = zero_indices[torch.randperm(zero_indices.size(0))][:num]
     
    combined_indices = torch.sort(torch.cat((zero_indices, one_indices)))
    
    selected_labels = labels[combined_indices.values]
    
    return combined_indices.values.tolist(), selected_labels 


    
def select_couples(y,type_):
    """ 
    ## type == 0 : select all the positions

    ## type == 1: let k = min(numbers of zeros, numbers of ones), select the first k zeros and the first k ones every time (always the same couples in each cluster)
           
    ## type == 2: let k = min(numbers of zeros, numbers of ones), select all the k elements of the minority class and  k + (#majority - #minority)/3 (always the same couples in each cluster)
        
    ## type == 3 : let k = min(numbers of zeros, numbers of ones), select all the k elements of the minority class and sample k elements from the majority one"""


    if type_ == 0:
                # all the position (unbalanced)
            positions, lab = all_pos(y)
    elif type_ == 1:
            # same 0 and 1 (balanced)
            positions, lab = fixed_and_balanced(y)
            
    elif type_ == 2:
            # not the same 0 and 1 (unbalanced)
            positions, lab = fixed_and_unbalanced(y)
    elif type_ == 3: 
            # same 0 and 1 + sampling (balanced)    
            positions, lab = sampled_balanced(y)
    else:
            raise ValueError("Type can be only  0, 1, 2 or 3")
    return positions, lab    

### ok

In [12]:
class PairModel2(nn.Module):
    def __init__(self):
        super().__init__()
        
        output_channels = 2 # it's a binary classification

      
        
        self.relu = nn.ReLU()
            
        # classificator
        self.linear0 = nn.Linear(2048, 1024, bias=False)
        self.bn0 = nn.BatchNorm1d(1024)
        self.dp0 = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(1024, 512, bias=False)
        self.bn1 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=0.3)
        self.linear3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dp3 = nn.Dropout(p=0.3)
        self.linear4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dp4 = nn.Dropout(p=0.3)
        self.linear5 = nn.Linear(64, output_channels)
        
    def forward(self, x_1, x_2):
       
        x_mult = x_1 * x_2 # sum the two elements of the couples
        x_sum = x_1 * x_2 # multiply the two elements of the couples
        x = torch.cat((x_mult, x_sum), dim = 1) 

        # classificator
        if x_1.shape[0] > 1:
            x = self.relu(self.bn0(self.linear0(x)))
            #x = self.dp0(x)
            x = self.relu(self.bn1(self.linear1(x)))
            x = self.dp1(x)
            x = self.relu(self.bn2(self.linear2(x)))
            x = self.dp2(x)
            x = self.relu(self.bn3(self.linear3(x)))
            x = self.dp3(x)
            x = self.relu(self.bn4(self.linear4(x)))
            x = self.dp4(x)
            x = self.linear5(x)

        # If we have only one pair, applying batch normalization doesn't make sense.
        else: 
            x = self.relu(self.linear0(x))
            #x = self.dp0(x)
            x = self.relu(self.linear1(x))
            x = self.dp1(x)
            x = self.relu(self.linear2(x))
            x = self.dp2(x)
            x = self.relu(self.linear3(x))
            x = self.dp3(x)
            x = self.relu(self.linear4(x))
            x = self.dp4(x)
            x = self.linear5(x)
                    
        return x


class Model2_mod(nn.Module):
    def __init__(self):
        super().__init__()

        self.ptc_net = Branch()
        self.pair_net = PairModel2()      

    def forward(self,cluster, positions, n_frags):
        
        cluster = cluster.double().to(device)

        # Salva l'ordine originale dei dati
        original_order = np.arange(len(cluster))
        # Esegui lo shuffle dei dati
        np.random.shuffle(original_order)
        cluster = cluster[original_order]

        # create an empty tensor that will collect the trasformed fragments
        cluster_transformed = torch.Tensor([]).to(device)

        sub_cluster_start = 0
        while sub_cluster_start < n_frags:
            sub_cluster_end = min(sub_cluster_start + 16, n_frags)
            cluster_subset = cluster[sub_cluster_start:sub_cluster_end]
            cluster_subset = cluster_subset.to(device)
            # apply the PCT
            point_clouds_transformed = self.ptc_net(cluster_subset)
            # append the results
            cluster_transformed = torch.cat((cluster_transformed, point_clouds_transformed), dim=0)
            sub_cluster_start += 16

        sorted_indices = np.argsort(original_order)
        cluster_transformed = cluster_transformed[sorted_indices]
        
        # given the positions and the transformed cluster create the couples
        indices = torch.triu_indices(n_frags, n_frags, offset=1)
    
        # Seleziona solo gli indici che corrispondono a posizioni desiderate
        selected_indices = indices[:, positions]
        
        # Estrai i tensori dai frammenti selezionati
        frags_a = cluster_transformed[selected_indices[0]]
        frags_b = cluster_transformed[selected_indices[1]] 

        frags_a = frags_a.double().to(device)
        frags_b = frags_b.double().to(device)
    
        outputs = self.pair_net(frags_a, frags_b)  
        return outputs, frags_a, frags_b

# Model

In [13]:
device=use_GPU()

NVIDIA GeForce RTX 4080 is available and being used


In [14]:
train = train.tolist()
val = val.tolist()
# We process one cluster at a time.
train_loader = DataLoader(train, batch_size=1)
val_loader = DataLoader(val, batch_size=1)

In [15]:
model = Model2_mod().to(device)
model.double()

Model2_mod(
  (ptc_net): Branch(
    (conv1): Conv1d(7, 64, kernel_size=(1,), stride=(1,), bias=False)
    (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (gather_local_0): Local_op(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (gather_local_1): Local_op(
      (conv1): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=Tru

In [22]:
W_stored = torch.load(r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\Check_points\\18_10_1616_1_vff.pt')  # run Not_sampled_0.0003 on w and B site

model.load_state_dict(W_stored)
#1012_164220_4+lr=0.0003+balanced_coupels.pt

<All keys matched successfully>

In [17]:
wandb.init(
      # Set the project where this run will be logged
      project="Modello_ottimizzato", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      notes = "modello ottimizzato senza cicli for ",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0001,
      "architecture": "Model2_mod",
      "epochs": 10,
      "weight_decay": 0.00005,
      "W_crossentropy":1,
      "W_contrastive":0,
      "type_of_couples": 1,
      "num_of batch" : len(train_loader),
      "seed": seed,
      "name_saved":'vff'  
      })
      
config = wandb.config

# Calcola i pesi delle classi in base alle percentuali
#weight_class_0 = 1.0 / 0.7  # Peso per la classe 0
#weight_class_1 = 1.0 / 0.3  # Peso per la classe 1

# Crea un tensore PyTorch con i pesi
#weight = torch.tensor([weight_class_0, weight_class_1], dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss().to(device)
contrast_criterion = ContrastiveLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
num_epochs = config.epochs
best_val_accuracy = 0.0 



#miner = miners.MultiSimilarityMiner()
#loss_func = losses.TripletMarginLoss()

In [23]:
checkpoint_dir = r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\Check_points'
results_dir = r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\results'

checkpoint_interval = 1  


for epoch in range(num_epochs):
    list_of_results=[]
    list_of_true=[]

    train_of_results=[]
    train_of_true=[]

    ######################################################################################
    ##                                      Train                                       ##
    ######################################################################################
    model.train() 

    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for batch_data in progress_bar:

        optimizer.zero_grad() 
        
        # Each cluster is composed by an adjacency matrix and the cluster of fragments
        matrix, cluster = batch_data
        
        #1 Let's convert the matrix into an y list and count the number of elements in the matrix
        matrix = matrix.numpy()
        ind=torch.triu_indices(len(matrix[0]),len(matrix[0]),offset =1)
        y=matrix[0][ind[0],ind[1]]
        n_frag_ = len(matrix[0])

        #2 select the positions and the relative labels that represent the pairs we want to extract
        positions, labels_ = select_couples(y,config.type_of_couples)
        labels_ = labels_.long().to(device)
        one_hot_labels_ = F.one_hot(labels_,2)
        train_of_true.append(labels_.tolist()) # append the true y


        #3 apply translation to the origin and random rotations
        cluster = apply_translation(cluster[0])
        cluster = apply_randomrotations(cluster)

        #4 input data to the model
        cluster_outputs_, frags_a, frags_b = model(cluster, positions, n_frag_)
        
        #5 calculate the cross entropy and the contrastive  losses
        loss_ = criterion(cluster_outputs_, one_hot_labels_.float())
        contrast_loss = contrast_criterion(frags_a, frags_b, labels_)
        loss = loss_ + config.W_contrastive*contrast_loss

        #6 backpropagation
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        #7 predict the y 
        _, predicted = torch.max(cluster_outputs_.data, 1)
        train_of_results.append(predicted.tolist()) # append the predicted y
        total_samples += one_hot_labels_.size(0)
        correct_predictions += (predicted == labels_).sum().item()
        progress_bar.set_postfix({'Loss': loss.item(), 'Accuracy': correct_predictions / total_samples}) # the progress bar tha shows for each cluster the live metrics
   
    #8 compute the metrics at the end of each epoch
    accuracy = correct_predictions / total_samples
    train_loss = total_loss/len(train_loader)
    
    #9 log the metrics and the model of the current epoch
    metrics_train = {"train_loss": train_loss, 
                       "accuracy": accuracy}
    wandb.log(metrics_train)
    
    current_time = datetime.datetime.now()
    checkpoint_name = f"{current_time.strftime('%d_%m_%H%M')}_{epoch+1}_{config.name_saved}.pt"    #################
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
    torch.save(model.state_dict(), checkpoint_path)

    wandb.run.log_artifact(checkpoint_path, name=checkpoint_name)



    ######################################################################################
    ##                                    Inference                                     ##
    ######################################################################################
 
    #                           #
    # COPPIE BILANCIATE E FISSE #
    #                           # 

    model.eval()  
    
    val_loss_ = 0.0
    v_contrast_loss= 0.0
    val_correct_predictions = 0
    val_total_samples = 0


    with torch.no_grad():
        for val_batch in val_loader:

            val_matrix, val_cluster = val_batch

            val_matrix = val_matrix.numpy()
            ind_val=torch.triu_indices(len(val_matrix[0]),len(val_matrix[0]),offset =1)
            y_val =val_matrix[0][ind_val[0],ind_val[1]]
            n_frag_val = len(val_matrix[0])

            # select the same number of 0 and 1 in the previous vector 
            val_positions,  val_labels_ = select_couples(y_val,1)
            val_labels_ = val_labels_.long().to(device)
            one_hot_labels_val_ = F.one_hot(val_labels_,2)
            list_of_true.append(val_labels_.tolist())

            val_cluster = apply_translation(val_cluster[0])
            val_cluster = apply_randomrotations(val_cluster)
            val_outputs_,  v_frags_a, v_frags_b = model(val_cluster,  val_positions, n_frag_val)
            
            val_loss_ += criterion(val_outputs_, one_hot_labels_val_.float()).item()
            v_contrast_loss += contrast_criterion(v_frags_a, v_frags_b, val_labels_).item() 
           
            val_loss = val_loss_ + config.W_contrastive*v_contrast_loss

            _, val_predicted = torch.max(val_outputs_.data, 1)
            list_of_results.append(val_predicted.tolist())
            val_total_samples += one_hot_labels_val_.size(0)
            val_correct_predictions += (val_predicted == val_labels_).sum().item()

        val_accuracy = val_correct_predictions / val_total_samples
        val_loss /= len(val_loader)
    
    val_metrics = {"val_loss": val_loss, 
                       "val_accuracy": val_accuracy}
    wandb.log(val_metrics)  


    #                  #
    # TUTTE LE COPPIE  #
    #                  # 

    model.eval()  
    
    val_loss_ = 0.0
    v_contrast_loss= 0.0
    val_correct_predictions = 0
    val_total_samples = 0


    with torch.no_grad():
        for val_batch in val_loader:

            val_matrix, val_cluster = val_batch

            val_matrix = val_matrix.numpy()
            ind_val=torch.triu_indices(len(val_matrix[0]),len(val_matrix[0]),offset =1)
            y_val =val_matrix[0][ind_val[0],ind_val[1]]
            n_frag_val = len(val_matrix[0])

            # select the same number of 0 and 1 in the previous vector 
            val_positions,  val_labels_ = select_couples(y_val,0)
            val_labels_ = torch.Tensor(val_labels_)
            val_labels_ = val_labels_.long().to(device)
            one_hot_labels_val_ = F.one_hot(val_labels_,2)
            list_of_true.append(val_labels_.tolist())

            val_cluster = apply_translation(val_cluster[0])
            val_cluster = apply_randomrotations(val_cluster)
            val_outputs_,  v_frags_a, v_frags_b = model(val_cluster,  val_positions, n_frag_val)
            
            val_loss_ += criterion(val_outputs_, one_hot_labels_val_.float()).item()
            v_contrast_loss += contrast_criterion(v_frags_a, v_frags_b, val_labels_).item() 
           
            val_loss_all = val_loss_ + config.W_contrastive*v_contrast_loss

            _, val_predicted = torch.max(val_outputs_.data, 1)
            list_of_results.append(val_predicted.tolist())
            val_total_samples += one_hot_labels_val_.size(0)
            val_correct_predictions += (val_predicted == val_labels_).sum().item()

        val_accuracy_all = val_correct_predictions / val_total_samples
        val_loss_all /= len(val_loader)
    
    all_the_values = {"val_loss_all": val_loss_all, 
                       "val_accuracy_all": val_accuracy_all}

    wandb.log(all_the_values)                   

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Loss unbalanced: {val_loss_all:.4f}, Val Accuracy unbalanced: {val_accuracy_all:.4f}')

###################################################### STORE THE RESULTS ###############################################################################################################

    result_path = os.path.join(results_dir, f"VAL+_{config.name_saved}_{epoch+1}")              #############################
    os.makedirs(result_path, exist_ok=True)

    with open(os.path.join(result_path, "list_of_true.pkl"), 'wb') as f:
        pickle.dump(list_of_true, f)

    with open(os.path.join(result_path, "list_of_results.pkl"), 'wb') as f:
        pickle.dump(list_of_results, f)

    train_result_path = os.path.join(results_dir, f"TRAIN+_{config.name_saved}_{epoch+1}")             ##################  
    os.makedirs(train_result_path, exist_ok=True)

    with open(os.path.join(train_result_path, "train_of_true.pkl" ), 'wb') as f:
        pickle.dump(train_of_true, f)

    with open(os.path.join(train_result_path, "train_of_results.pkl"), 'wb') as f:
        pickle.dump(train_of_results, f)    

Epoch 1/10:   0%|          | 0/1348 [00:00<?, ?it/s]

Epoch [1/10], Train Loss: 0.4839, Train Accuracy: 0.7951, Val Loss: 0.5187, Val Accuracy: 0.7845, Val Loss unbalanced: 1.1759, Val Accuracy unbalanced: 0.4966


Epoch [2/10], Train Loss: 0.4719, Train Accuracy: 0.8029, Val Loss: 0.5180, Val Accuracy: 0.7715, Val Loss unbalanced: 1.1938, Val Accuracy unbalanced: 0.5357


Epoch [3/10], Train Loss: 0.4653, Train Accuracy: 0.8051, Val Loss: 0.5286, Val Accuracy: 0.7883, Val Loss unbalanced: 1.3301, Val Accuracy unbalanced: 0.4953


Epoch [4/10], Train Loss: 0.4593, Train Accuracy: 0.8066, Val Loss: 0.5163, Val Accuracy: 0.7772, Val Loss unbalanced: 1.2454, Val Accuracy unbalanced: 0.5262


Epoch [5/10], Train Loss: 0.4517, Train Accuracy: 0.8110, Val Loss: 0.5237, Val Accuracy: 0.7848, Val Loss unbalanced: 1.3264, Val Accuracy unbalanced: 0.5101


Epoch [6/10], Train Loss: 0.4440, Train Accuracy: 0.8131, Val Loss: 0.5364, Val Accuracy: 0.7747, Val Loss unbalanced: 1.4049, Val Accuracy unbalanced: 0.5325


KeyboardInterrupt: 

In [ ]:
wandb.finish()

accuracy,█▄▁
train_loss,▁▅█
val_accuracy,█▁▁
val_accuracy_all,▇█▁
val_loss,▁█▇
val_loss_all,█▅▁
accuracy,0.66253
train_loss,0.63715
val_accuracy,0.68106
val_accuracy_all,0.42655
val_loss,0.63491


In [20]:
model.eval()  
    
val_loss_ = 0.0
v_contrast_loss= 0.0
val_correct_predictions = 0
val_total_samples = 0


with torch.no_grad():
        for val_batch in val_loader:

            val_matrix, val_cluster = val_batch

            val_matrix = val_matrix.numpy()
            ind_val=torch.triu_indices(len(val_matrix[0]),len(val_matrix[0]),offset =1)
            y_val =val_matrix[0][ind_val[0],ind_val[1]]
            n_frag_val = len(val_matrix[0])

            # select the same number of 0 and 1 in the previous vector 
            val_positions,  val_labels_ = select_couples(y_val,0)
            val_labels_ = torch.Tensor(val_labels_)
            val_labels_ = val_labels_.long().to(device)
            one_hot_labels_val_ = F.one_hot(val_labels_,2)
            list_of_true.append(val_labels_.tolist())

            val_cluster = apply_translation(val_cluster[0])
            val_cluster = apply_randomrotations(val_cluster)
            val_outputs_,  v_frags_a, v_frags_b = model(val_cluster,  val_positions, n_frag_val)
            
            val_loss_ += criterion(val_outputs_, one_hot_labels_val_.float()).item()
            v_contrast_loss += contrast_criterion(v_frags_a, v_frags_b, val_labels_).item() 
           
            val_loss_all = val_loss_ + config.W_contrastive*v_contrast_loss

            _, val_predicted = torch.max(val_outputs_.data, 1)
            list_of_results.append(val_predicted.tolist())
            val_total_samples += one_hot_labels_val_.size(0)
            val_correct_predictions += (val_predicted == val_labels_).sum().item()

        val_accuracy_all = val_correct_predictions / val_total_samples
        val_loss_all  /= len(val_loader)

    
all_the_values = {"val_loss_all": val_loss_all, 
                       "val_accuracy_all": val_accuracy_all}

wandb.log(all_the_values)                   

print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Loss unbalanced: {val_loss_all:.4f}, Val Accuracy unbalanced: {val_accuracy_all:.4f}')


Epoch [1/10], Train Loss: 0.5223, Train Accuracy: 0.7734, Val Loss: 285.9390, Val Accuracy: 0.7573, Val Loss unbalanced: 0.9756, Val Accuracy unbalanced: 0.5558


In [21]:
result_path = os.path.join(results_dir, f"VAL+_{config.name_saved}_{epoch+1}")              #############################
os.makedirs(result_path, exist_ok=True)

with open(os.path.join(result_path, "list_of_true.pkl"), 'wb') as f:
        pickle.dump(list_of_true, f)

with open(os.path.join(result_path, "list_of_results.pkl"), 'wb') as f:
        pickle.dump(list_of_results, f)

train_result_path = os.path.join(results_dir, f"TRAIN+_{config.name_saved}_{epoch+1}")             ##################  
os.makedirs(train_result_path, exist_ok=True)

with open(os.path.join(train_result_path, "train_of_true.pkl" ), 'wb') as f:
        pickle.dump(train_of_true, f)

with open(os.path.join(train_result_path, "train_of_results.pkl"), 'wb') as f:
        pickle.dump(train_of_results, f)    